In [1]:
import subprocess
import sys

def install_playwright_browsers():
    subprocess.check_call([sys.executable, "-m", "playwright", "install"])

# Executar a instalação
install_playwright_browsers()

In [2]:
import asyncio
import re
import pandas as pd
from playwright.async_api import async_playwright
import nest_asyncio

# Funções de extração de dados
def extrair_titulo_com_ano(linha):
    match = re.search(r'(\d{4})', linha)
    return linha[:match.end()].strip() if match else linha.strip()

def extrair_rentabilidade(linha):
    match = re.search(r'\d{4}\s(.*?)\sR\$', linha)
    return match.group(1).strip() if match else None

def extrair_investimento_minimo(linha):
    match = re.search(r'R\$\s(.*?)\sR\$', linha)
    return match.group(1).strip() if match else None

def extrair_preco_unitario(linha):
    match = re.search(r'R\$\s.*?\sR\$\s(.*?)\s', linha)
    return match.group(1).strip() if match else None

def extrair_data_vencimento(linha):
    match = re.search(r'\s(\d{2}/\d{2}/\d{4})$', linha)
    return match.group(1).strip() if match else None

def extrair_tipo_e_rentabilidade(rentabilidade):
    if "+" in rentabilidade:
        tipo, rentabilidade = rentabilidade.split(" + ", maxsplit=1)
        return tipo.strip(), rentabilidade.strip()
    return "PREFIXADO", rentabilidade.strip()

async def processar_dados(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        await page.wait_for_selector(".td-invest-table__card")
        
        cartoes = await page.query_selector_all(".td-invest-table__card")
        dados_cartao = [await cartao.inner_text() for cartao in cartoes]
                # Find the index of the last 'Simule'
        last_simule_index = max(i for i, item in enumerate(dados_cartao) if 'Simule' in item)
        dados_cartao = dados_cartao[:last_simule_index + 1]
        
        dados_cartao = [" ".join(texto.strip().replace("\n", " ").replace("\t", " ").split()) for texto in dados_cartao]
        dados_cartao = [texto.replace(" Simule", "") for texto in dados_cartao]

        await browser.close()

    # O restante do processamento de dados permanece o mesmo
    titulos = [extrair_titulo_com_ano(linha) for linha in dados_cartao]
    rentabilidades = [extrair_rentabilidade(linha) for linha in dados_cartao]
    investimentos_minimos = [extrair_investimento_minimo(linha) for linha in dados_cartao]
    precos_unitarios = [extrair_preco_unitario(linha) for linha in dados_cartao]
    datas_vencimento = [extrair_data_vencimento(linha) for linha in dados_cartao]

    df = pd.DataFrame({
        "Título": titulos,
        "Rentabilidade": rentabilidades,
        "Investimento Mínimo": investimentos_minimos,
        "Preço Unitário": precos_unitarios,
        "Data de Vencimento": datas_vencimento,
    })

    df.dropna(inplace=True)

    tipos_e_rentabilidades = df["Rentabilidade"].apply(extrair_tipo_e_rentabilidade)
    df["Tipo"], df["Rentabilidade"] = zip(*tipos_e_rentabilidades)

    return df

# Criar uma variável global para armazenar o resultado
df_resultado_global = None

# Função para executar o código assíncrono
async def main():
    global df_resultado_global
    url = "https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm"
    df_resultado_global = await processar_dados(url)
    display(df_resultado_global)


# Executar o código assíncrono

nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(main())


,Título,Rentabilidade,Investimento Mínimo,Preço Unitário,Data de Vencimento,Tipo
0,TESOURO PREFIXADO 2027,"14,89%","7,68","768,09",01/01/2027,PREFIXADO
1,TESOURO PREFIXADO 2031,"14,72%","4,46","446,65",01/01/2031,PREFIXADO
2,TESOURO PREFIXADO com juros semestrais 2035,"14,70%","7,80","780,55",01/01/2035,PREFIXADO
3,TESOURO SELIC 2027,"0,0396%","159,75","15.975,87",01/03/2027,SELIC
4,TESOURO SELIC 2029,"0,1050%","159,21","15.921,20",01/03/2029,SELIC
5,TESOURO IPCA+ 2029,"7,55%","32,32","3.232,58",15/05/2029,IPCA
6,TESOURO IPCA+ 2035,"7,60%","20,83","2.083,67",15/05/2035,IPCA
7,TESOURO IPCA+ 2045,"7,42%","10,40","1.040,23",15/05/2045,IPCA
8,TESOURO IPCA+ com juros semestrais 2035,"7,59%","39,92","3.992,81",15/05/2035,IPCA
9,TESOURO IPCA+ com juros semestrais 2040,"7,36%","40,03","4.003,84",15/08/2040,IPCA


In [3]:
# Agora você pode acessar o DataFrame globalmente no notebook
display(df_resultado_global)

,Título,Rentabilidade,Investimento Mínimo,Preço Unitário,Data de Vencimento,Tipo
0,TESOURO PREFIXADO 2027,"14,89%","7,68","768,09",01/01/2027,PREFIXADO
1,TESOURO PREFIXADO 2031,"14,72%","4,46","446,65",01/01/2031,PREFIXADO
2,TESOURO PREFIXADO com juros semestrais 2035,"14,70%","7,80","780,55",01/01/2035,PREFIXADO
3,TESOURO SELIC 2027,"0,0396%","159,75","15.975,87",01/03/2027,SELIC
4,TESOURO SELIC 2029,"0,1050%","159,21","15.921,20",01/03/2029,SELIC
5,TESOURO IPCA+ 2029,"7,55%","32,32","3.232,58",15/05/2029,IPCA
6,TESOURO IPCA+ 2035,"7,60%","20,83","2.083,67",15/05/2035,IPCA
7,TESOURO IPCA+ 2045,"7,42%","10,40","1.040,23",15/05/2045,IPCA
8,TESOURO IPCA+ com juros semestrais 2035,"7,59%","39,92","3.992,81",15/05/2035,IPCA
9,TESOURO IPCA+ com juros semestrais 2040,"7,36%","40,03","4.003,84",15/08/2040,IPCA
